In [1]:
# Install sentence-transformers
!pip install -q sentence-transformers

In [1]:
# Import libraries
from langchain_community.graphs import Neo4jGraph
from graph_rag_embeddings import EntityEmbeddings
import numpy as np

/home/duo/miniconda3/envs/rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Setup Neo4j connection
NEO4J_URI = "neo4j+s://0c367113.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "gTO1K567hBLzkRdUAhhEb-UqvBjz0i3ckV3M9v_-Nio"

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

print("Kết nối Neo4j thành công!")

/tmp/ipykernel_80588/502800272.py:6: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the `langchain-neo4j package and should be used instead. To use it run `pip install -U `langchain-neo4j` and import as `from `langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


Kết nối Neo4j thành công!


## Init Embedding Model

In [3]:
embeddings = EntityEmbeddings(
    graph=graph,
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)

print(f"\nModel: {embeddings.model_name}")
print(f"Embedding dimension: {embeddings.embedding_dim}")

Loading embedding model: sentence-transformers/paraphrase-multilingual-mpnet-base-v2
Model loaded! Embedding dimension: 768

Model: sentence-transformers/paraphrase-multilingual-mpnet-base-v2
Embedding dimension: 768


## Check Current State

In [6]:
# Check embedding coverage trước khi generate
coverage = embeddings.check_embedding_coverage()

print("=== EMBEDDING COVERAGE ===")
print(f"Total entities: {coverage['total_entities']:,}")
print(f"Entities with embeddings: {coverage['entities_with_embeddings']:,}")
print(f"Coverage: {coverage['coverage_percent']:.1f}%")

print("\nCoverage by type:")
for row in coverage['coverage_by_type']:
    print(f"  {row['type']}: {row['with_embedding']}/{row['total']} ({row['coverage_percent']:.1f}%)")

=== EMBEDDING COVERAGE ===
Total entities: 1,726
Entities with embeddings: 1,726
Coverage: 100.0%

Coverage by type:
  LOCATION: 669/669 (100.0%)
  ORGANIZATION: 462/462 (100.0%)
  PERSON: 398/398 (100.0%)
  EVENT: 79/79 (100.0%)
  TIME: 49/49 (100.0%)
  PRODUCT: 26/26 (100.0%)
  BOOK: 11/11 (100.0%)
  OBJECT: 7/7 (100.0%)
  NUMBER: 6/6 (100.0%)
  DOCUMENT: 5/5 (100.0%)
  PERCENTAGE: 2/2 (100.0%)
  ENTITY: 2/2 (100.0%)
  ROUTE: 2/2 (100.0%)
  PERSON_GROUP: 2/2 (100.0%)
  GROUP: 1/1 (100.0%)
  ENEMY: 1/1 (100.0%)
  LANGUAGE: 1/1 (100.0%)
  GOVERNMENT: 1/1 (100.0%)
  DATE: 1/1 (100.0%)
  POSITION: 1/1 (100.0%)


## Generate Embeddings for All Entities

In [5]:
# Generate embeddings cho tất cả entities
stats = embeddings.generate_and_store_all_embeddings(batch_size=32)

print("\n=== STATISTICS ===")
print(f"Total entities: {stats['total']}")
print(f"Successfully embedded: {stats['success']}")
print(f"Failed: {stats['failed']}")
print(f"Embedding dimension: {stats['embedding_dim']}")

Starting embedding generation...


Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. id is deprecated. It is replaced by elementId or consider using an application-generated id.', position=<SummaryInputPosition line=7, column=16, offset=180>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 180, 'line': 7, 'column': 16}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: '\n        MATCH (n)\n        WHERE n.name IS NOT NULL\n        RETURN n.name as name,\n               labels(n)[0] as type,\n               n.description as description,\n               id(n) as node_id,\n               n.first_seen_page as first_seen_page,\n               n.first_seen_chapter as first_seen_cha

Found 1726 entities in graph
Creating entity texts...
Generating embeddings for 1726 entities...


Batches: 100%|██████████| 54/54 [00:02<00:00, 21.34it/s]


Storing embeddings to Neo4j...


 63%|██████▎   | 1083/1726 [03:37<3:14:22, 18.14s/it]

Error storing embedding for Thái Văn Lung: Failed to read from defunct connection IPv4Address(('si-0c367113-f6e4.production-orch-0703.neo4j.io', 7687)) (ResolvedIPv4Address(('34.124.169.171', 7687)))


[#AE7A]  _: <CONNECTION> error: Failed to read from defunct connection ResolvedIPv4Address(('34.124.169.171', 7687)) (ResolvedIPv4Address(('34.124.169.171', 7687))): OSError('No data')
Unable to retrieve routing information
Transaction failed and will be retried in 1.059983884582644s (Unable to retrieve routing information)
100%|██████████| 1726/1726 [05:48<00:00,  4.95it/s]  


EMBEDDINGS GENERATED!
Success: 1725/1726
Failed: 1
Embedding dimension: 768

=== STATISTICS ===
Total entities: 1726
Successfully embedded: 1725
Failed: 1
Embedding dimension: 768


## Verify Embeddings

In [6]:
# Check coverage sau khi generate
coverage_after = embeddings.check_embedding_coverage()

print("=== EMBEDDING COVERAGE (AFTER) ===")
print(f"Total entities: {coverage_after['total_entities']:,}")
print(f"Entities with embeddings: {coverage_after['entities_with_embeddings']:,}")
print(f"Coverage: {coverage_after['coverage_percent']:.1f}%")

# Sample some entities với embeddings
sample_entities = embeddings.get_entities_with_embeddings(limit=5)

print("\n=== SAMPLE ENTITIES WITH EMBEDDINGS ===")
for ent in sample_entities:
    emb = ent['embedding']
    print(f"\n{ent['name']} ({ent['type']})")
    print(f"  Description: {ent['description'][:60]}...")
    print(f"  Embedding: [{emb[0]:.4f}, {emb[1]:.4f}, ..., {emb[-1]:.4f}] (dim={len(emb)})")

=== EMBEDDING COVERAGE (AFTER) ===
Total entities: 1,725
Entities with embeddings: 1,725
Coverage: 100.0%

=== SAMPLE ENTITIES WITH EMBEDDINGS ===

vùng hữu ngạn sông Hồng (LOCATION)
  Description: ...
  Embedding: [-0.0801, -0.0121, ..., -0.0235] (dim=768)

Liên khu Việt Bắc (LOCATION)
  Description: Khu vực lãnh thổ...
  Embedding: [0.0792, 0.1510, ..., 0.0326] (dim=768)

Liên khu (LOCATION)
  Description: ...
  Embedding: [-0.1122, -0.0189, ..., -0.1050] (dim=768)

Tháng 3-1949 (TIME)
  Description: ...
  Embedding: [-0.1158, 0.2067, ..., -0.0530] (dim=768)

30.000 mẫu (TIME)
  Description: ...
  Embedding: [-0.0808, -0.0189, ..., -0.1431] (dim=768)


## Test Semantic Search

Thử search entities bằng natural language queries

In [8]:
# Test queries về lịch sử Việt Nam
test_queries = [
    "lãnh đạo cách mạng Việt Nam",
    "chiến dịch quân sự quan trọng",
    "thủ đô",
    "tổ chức chính trị",
    "cuộc kháng chiến chống Pháp"
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"Query: '{query}'")
    print(f"{'='*60}")
    
    results = embeddings.semantic_search(query, top_k=5)
    
    for i, result in enumerate(results, 1):
        print(f"\n{i}. {result['name']} ({result['type']})")
        print(f"   Similarity: {result['similarity']:.4f}")
        if result['description']:
            print(f"   Description: {result['description'][:80]}...")


Query: 'lãnh đạo cách mạng Việt Nam'

1. Cách mạng Việt Nam (ORGANIZATION)
   Similarity: 0.8458

2. Việt Nam dân chủ cộng hòa (ORGANIZATION)
   Similarity: 0.8212
   Description: Nhà nước...

3. Việt Nam dân chủ cộng hòa (LOCATION)
   Similarity: 0.8212
   Description: Tên gọi cũ của Việt Nam hiện nay...

4. Việt Minh (ORGANIZATION)
   Similarity: 0.8196
   Description: Chiến dịch giải phóng miền Nam Việt Nam...

5. Việt Nam Cách mạng đồng minh Hội (ORGANIZATION)
   Similarity: 0.8175

Query: 'chiến dịch quân sự quan trọng'

1. phong tỏa (EVENT)
   Similarity: 0.8578
   Description: Hoạt động quân sự...

2. Chiến dịch Biên giới (EVENT)
   Similarity: 0.8533
   Description: Chiến dịch quân sự...

3. chiến tranh du kích (EVENT)
   Similarity: 0.8375
   Description: Hoạt động quân sự phi tập trung...

4. Thập vạn đại sơn (EVENT)
   Similarity: 0.8363
   Description: Chiến dịch quân sự...

5. giải phóng (EVENT)
   Similarity: 0.8138
   Description: Hoạt động quân sự...

Query: 'thủ đô'



## Test Search by Entity Type

In [9]:
# Search chỉ trong loại entities cụ thể
query = "lãnh đạo quân sự"

print(f"Query: '{query}'\n")

# Search trong PERSON
print("=== Search in PERSON ===")
person_results = embeddings.semantic_search(query, top_k=5, filter_types=['PERSON'])
for i, r in enumerate(person_results, 1):
    print(f"{i}. {r['name']} - similarity: {r['similarity']:.4f}")

# Search trong ORGANIZATION
print("\n=== Search in ORGANIZATION ===")
org_results = embeddings.semantic_search(query, top_k=5, filter_types=['ORGANIZATION'])
for i, r in enumerate(org_results, 1):
    print(f"{i}. {r['name']} - similarity: {r['similarity']:.4f}")

Query: 'lãnh đạo quân sự'

=== Search in PERSON ===
1. Tường Giới Thạch - similarity: 0.8874
2. Tướng Lư Hán - similarity: 0.7442
3. Đàng - similarity: 0.7204
4. Hoàng Văn Thái - similarity: 0.7001
5. Thiếu tướng Trần Đại Nghĩa - similarity: 0.6946

=== Search in ORGANIZATION ===
1. Bộ Chi huy - similarity: 0.8137
2. Bộ Tổng chi huy - similarity: 0.8037
3. Ban chi huy mặt trận Đông Khê - similarity: 0.7682
4. Tổng Tư lệnh - similarity: 0.7655
5. Quân sự ủy viên hội - similarity: 0.7393


## Find Similar Entities

Tìm entities tương tự với một entity cho trước

In [10]:
# Chọn một entity để tìm similar (thay tên phù hợp với data)
target_entity = "Hồ Chí Minh"  # Thay đổi tên này

print(f"=== ENTITIES SIMILAR TO: {target_entity} ===")

similar = embeddings.find_similar_entities(target_entity, top_k=10)

for i, entity in enumerate(similar, 1):
    print(f"\n{i}. {entity['name']} ({entity['type']})")
    print(f"   Similarity: {entity['similarity']:.4f}")
    if entity['description']:
        print(f"   Description: {entity['description'][:80]}...")

=== ENTITIES SIMILAR TO: Hồ Chí Minh ===

1. Nguyễn Chí Thanh (PERSON)
   Similarity: 0.9125
   Description: Chủ nhiệm Tổng cục Chính trị...

2. Nguyễn Văn Tư (PERSON)
   Similarity: 0.9062
   Description: Chủ tịch Chợ Lớn...

3. Hoàng Minh Giám (PERSON)
   Similarity: 0.9056
   Description: Đại diện cho Chính phủ Việt Nam...

4. Nguyễn Hải Thần (PERSON)
   Similarity: 0.8600
   Description: Phó Chủ tịch...

5. Chủ tịch nước (ORGANIZATION)
   Similarity: 0.8423
   Description: Việt Nam Dân chủ Cộng hòa...

6. Nguyễn Minh Phụng (PERSON)
   Similarity: 0.8200

7. Nguyễn Ngọc Minh (PERSON)
   Similarity: 0.8176

8. HỒ Chí Minh (PERSON)
   Similarity: 0.8132

9. Nguyễn Đình Thi (PERSON)
   Similarity: 0.8125
   Description: Ủy viên chính thức Ban thường trực Quốc hội...

10. Nguyễn Thị Thục Viên (PERSON)
   Similarity: 0.8047
   Description: Ủy viên dự khuyết Ban thường trực Quốc hội...


## Test Different Query Types

In [ ]:
# Test với các loại câu hỏi khác nhau
complex_queries = [
    {
        "query": "Ai là người lãnh đạo phong trào cách mạng giải phóng dân tộc?",
        "description": "Question về leader"
    },
    {
        "query": "Các trận đánh lớn trong chiến tranh",
        "description": "Question về military events"
    },
    {
        "query": "Những địa điểm chiến lược quan trọng",
        "description": "Question về locations"
    },
    {
        "query": "Các tổ chức chính trị và quân sự",
        "description": "Question về organizations"
    }
]

for q_info in complex_queries:
    query = q_info['query']
    desc = q_info['description']
    
    print(f"\n{'='*70}")
    print(f"{desc}")
    print(f"Query: '{query}'")
    print(f"{'='*70}")
    
    results = embeddings.semantic_search(query, top_k=3)
    
    for i, r in enumerate(results, 1):
        print(f"\n  {i}. {r['name']} ({r['type']}) - {r['similarity']:.4f}")
        if r['description']:
            print(f"     {r['description'][:100]}...")

## Compare with Exact Name Search

So sánh semantic search vs exact name search

In [ ]:
from graph_rag_query import GraphRAGQuery

# Initialize graph query
rag_query = GraphRAGQuery(graph)

# Test case: User không biết tên chính xác
print("=== SCENARIO: User hỏi về 'chủ tịch' nhưng không biết tên ===")

search_term = "chủ tịch"

print(f"\nEXACT NAME SEARCH (Bước 1):")
exact_results = rag_query.search_entities_by_name(search_term, limit=5)
print(f"   Found {len(exact_results)} results")
for r in exact_results[:3]:
    print(f"   - {r['name']} ({r['type']})")

print(f"\nSEMANTIC SEARCH (Bước 2):")
semantic_results = embeddings.semantic_search(search_term, top_k=5)
print(f"   Found {len(semantic_results)} results")
for r in semantic_results[:3]:
    print(f"   - {r['name']} ({r['type']}) - similarity: {r['similarity']:.4f}")

print("\n Semantic search tìm được entities liên quan ngay cả khi không có từ khóa trong tên!")

## Update Embeddings for New Entities

Nếu add thêm entities mới sau này

In [ ]:
# Check và update embeddings cho entities mới
update_stats = embeddings.update_embeddings_for_new_entities()

print(f"\nNew embeddings generated: {update_stats['new_embeddings']}")
print(f"Total entities checked: {update_stats['total']}")